In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo
import mysql.connector as SQLC
import mysql.connector
from datetime import date

### Start povoation:

In [2]:
#Extrair ids:

import time
from datetime import datetime


data_e_hora_atuais = datetime.now()

query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

import requests, sys, json
content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    
import re
existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
numero_genes= len(n_genes)
print(n_genes)




escolha o que quer pesquisar: irs1
escolha o nº de genes que quer obter (máximo 20): 10
['3667', '3667', '16367', '25467', '559281', '3077574', '100512686', '538598', '459985', '100607873']


In [44]:
#Povoação "History"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor_2 = DataBase.cursor()
try:
    sql= "INSERT INTO History (search, Genes_number_input, Day)   VALUES (%s, %s, %s)"
    val=(query, numero_genes, data_e_hora_atuais)
    Cursor_2.execute(sql,val)
    
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
    search_id =[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor_2 = DataBase.cursor()
try:
    sql= "Select ID_search FROM History"
    Cursor_2.execute(sql)
    for row in Cursor_2:
        #print(row)
        search_id.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div= ', '.join(search_id)
h= div.replace("(",'')
hh= h.replace(",)",'')
SEARCH_ID= hh.split(', ')
Hist= SEARCH_ID[-1]

In [3]:
#Extrair ids NCBI:

Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    #print(info.id)
print (Ids)

C:\Users\rodri\anaconda3\lib\site-packages\Bio\Entrez\__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


['X03907.1', 'AA928418.1', 'AAADR0035516.1', 'L20655.1', 'AAADU0045050.1']


In [4]:
import time
def url_get_id(i):
    url_list= [ ]
    for id in i:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
link_genebank= url_get_id(Ids)
print(link_genebank)

['https://www.ncbi.nlm.nih.gov/nuccore/X03907.1', 'https://www.ncbi.nlm.nih.gov/nuccore/AA928418.1', 'https://www.ncbi.nlm.nih.gov/nuccore/AAADR0035516.1', 'https://www.ncbi.nlm.nih.gov/nuccore/L20655.1', 'https://www.ncbi.nlm.nih.gov/nuccore/AAADU0045050.1']


In [63]:
#Extrair description NCBI:

database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    #print(info.id)
    #print(info.description)
    description.append(info.description)
    print (description)

['A.thaliana gene (LHCP AB 165) for chlorophyll a/b binding protein']
['A.thaliana gene (LHCP AB 165) for chlorophyll a/b binding protein', "on49g12.s1 NCI_CGAP_Co8 Homo sapiens cDNA clone IMAGE:1560070 3' similar to gb:L06105 FARNESYL-DIPHOSPHATE FARNESYLTRANSFERASE (HUMAN), mRNA sequence"]
['A.thaliana gene (LHCP AB 165) for chlorophyll a/b binding protein', "on49g12.s1 NCI_CGAP_Co8 Homo sapiens cDNA clone IMAGE:1560070 3' similar to gb:L06105 FARNESYL-DIPHOSPHATE FARNESYLTRANSFERASE (HUMAN), mRNA sequence", 'Mus musculus strain C57BL/6J']
['A.thaliana gene (LHCP AB 165) for chlorophyll a/b binding protein', "on49g12.s1 NCI_CGAP_Co8 Homo sapiens cDNA clone IMAGE:1560070 3' similar to gb:L06105 FARNESYL-DIPHOSPHATE FARNESYLTRANSFERASE (HUMAN), mRNA sequence", 'Mus musculus strain C57BL/6J', 'Simian T-cell lymphotropic virus type I p24 protein (gag) gene, partial cds']
['A.thaliana gene (LHCP AB 165) for chlorophyll a/b binding protein', "on49g12.s1 NCI_CGAP_Co8 Homo sapiens cDNA clone

In [64]:
#organismos

Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Organismos.append(info.annotations['organism'])
print(Organismos)

['Arabidopsis thaliana', 'Homo sapiens', 'Mus musculus', 'Simian T-lymphotropic virus 1', 'Mus musculus']


In [50]:
#Biopython
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:10],'...',info.seq[-10:])
    print()
    seq.append(info.seq[0:10])

AL329769.1
GGGACAGATC ... CCCCANTCTT

X61083.1
AACTCCTTCC ... GAGGCCGCAA

U02356.1
CCCCGAGGCC ... GGTTGGAGAC

AASW01042081.1
ATTTTGTCAT ... GAATCCACCA

Y00554.1
GTCGACAACG ... CTTGTCGCTT

EL279606.1
GAAAGGGAAT ... ACAGAAGACA

EC941813.1
TTGATTTCAT ... GGGTCCCTTG

X61884.1
GGCGATGTGT ... ACATGTGATG

AAADZ0085841.1
GGGGTGTGGC ... GTGGATTTGC



In [65]:
Adenina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('A')
    perc= int(count/len(c)*100)
    Adenina.append(perc)
print (Adenina)

[25, 28, 25, 28, 5]


In [66]:
Citosina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('C')
    perc= int(count/len(c)*100)
    Citosina.append(perc)
print (Citosina)

[24, 21, 30, 30, 30]


In [67]:
Guanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('G')
    perc= int(count/len(c)*100)
    Guanina.append(perc)
print (Guanina)

[24, 23, 25, 19, 50]


In [68]:
Timina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('T')
    perc= int(count/len(c)*100)
    Timina.append(perc)
print (Timina)

[25, 27, 20, 21, 15]


In [69]:
data=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    data.append(info.annotations['date'])
print(data)

['20-FEB-1995', '06-JAN-2011', '20-SEP-2005', '13-MAR-1994', '20-SEP-2005']


In [70]:
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
tam=[]
for info in records:
    tam.append(len(info.seq))

print(tam)

[999, 725, 20, 92, 20]


In [71]:
#Permitir a povoação em uma coluna de cada vez:

import mysql.connector as SQLC
import mysql.connector

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try: 
    Cursor.execute("SET GLOBAL sql_mode = ''")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [73]:
#Povoação "Gene"

import mysql.connector as SQLC
import mysql.connector

#tab = ("INSERT INTO Gene (ID_genebank) VALUES ('{}')".format(value)) # %s overwrites the same value

des = (description)
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(Ids):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO Gene (ID_genebank, Description, Organism, length, Adenina, Citosina, Guanina, Timina, Link, ID_search) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val=(value, des[index], Organismos[index], tam[index], Adenina[index], Citosina[index], Guanina[index], Timina[index], link_genebank[index], Hist)
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [16]:
# buscar info pubmed

import requests
content_id = []
for url in url_get_id(Ids):
    r_id = requests.get(url)
    content_id.append(r_id.content)
    
import requests
from bs4 import BeautifulSoup
listas=[]
# Parsing the HTML
for c in content_id:
    soup_id = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

    id = ""
    url = ""
    for line in lines:
        #print(line)
        #if 'name' in line.attrs:
        #    print(line.attrs['name'])
        if 'content' in line.attrs:
            # print(line.attrs['content'])		
            id = line.attrs['content']

    if id:
        url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)

    r2 = requests.get(url)
    r4= r2.content.decode('utf-8')
    #r4= str(r2.content)
    listas.append(r4)
cc= ', '.join(listas)
er= cc.replace('//','')
final= er.split(', ')
#print (er)

output_dict = {}
for x in final:
    #print (x)
    version = re.search(r'VERSION\s+(.*?)\s', x)
    pubmed = re.search(r'PUBMED\s+(.*?)\s', x)
    if version:
#         print(version.group(1))
        versionf=version.group(1)
        output_dict.setdefault(version.group(1), [])
    if pubmed:
#         print(pubmed.group(1))
        output_dict[versionf].append(pubmed.group(1))
#     else:
       # output_dict[version.group(1)].append("N/A")

#print(output_dict)

id_ncbii = []
ID_PUB = []
for key, vals in output_dict.items():
    if vals:
        for val in vals:
            id_ncbii.append(key)
            ID_PUB.append(val)
    else:
        id_ncbii.append(key)
        ID_PUB.append("N/A")
    
new_list_ = []
seen = set()
for item in ID_PUB:
    if item not in seen:
        seen.add(item)
        new_list_.append(item)

so_ids_pubmed = [val for key, vals in output_dict.items() for val in vals]
so_ids_ncbi = list(output_dict.keys())

       
titles= []
authors=[]
source=[]
affiliation=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= new_list_
counter = 0
for i in idlist:
    if i!= "N/A":
        handle = Entrez.efetch(db=database, id=i, rettype="medline", retmode="text") 
        records = Medline.parse(handle)
        for info in records:
            #print(f'id: {i}')
            #print("title:", info.get("TI", "-"))
            titles.append(info.get("TI", "-"))
            #print('authors: ', info.get("AU", "-"))
            authors_string = info.get("AU", "-")
            if len(authors_string) > 5:
                authors_h = authors_string[0:5]
                authors.append(authors_h)
            else:
                authors.append(authors_string) 
            #print("source:", info.get("SO", "-"))
            source.append(info.get("SO", "-"))
            #print("affiliation: ", info.get("AD", "N/A") ) # FAZER O IF/ELSE PARA O CASO DE NÃO TER AFFILIATION
            affiliation_string= info.get("AD", ["N/A"])
            if len(affiliation_string) > 5:
                affiliation_h = affiliation_string[0:5]
                affiliation.append(affiliation_h)
            else:
                affiliation.append(affiliation_string)
            #print()
            counter += 1
    else:
        #print(f'id: {i} - N/A')
        titles.append("N/A")
        authors.append(["N/A"])
        source.append("N/A")
        affiliation.append(["N/A"])
        #print()
        counter += 1
        
#agrupar titles
titles = [ [title] for title in titles]

# agrupar dois
doi_list = []
for x in source:
    match = re.search("doi: (.*)", x)
    if match:
        doi_list.append(match.group(1))
    else:
        doi_list.append("N/A")
        
#agrupar authors
id_authors_dict = {i: authors[counter] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}        
        
pubmed_list = []
authors_list = []
for key, vals in id_authors_dict.items():
    if vals:
        for val in vals:
            pubmed_list.append(key)
            authors_list.append(val)

new_list_authors= []
seen = set()
for item in authors_list:
    if item not in seen:
        seen.add(item)
        new_list_authors.append(item)
        

#agrupar affi
single_affiliation_list = []
for i in affiliation:
    single_affiliation_list.extend(i)
    

id_affiliation_dict = {i: [single_affiliation_list[counter]] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}

pubmed_affi_list = []
affi_list = []
for key, vals in id_affiliation_dict.items():
    if vals:
        for val in vals:
            pubmed_affi_list.append(key)
            affi_list.append(val)
    else:
        pubmed_affi_list.append(key)
        affi_list.append(["N/A"])

new_list_affi= []
seen = set()
for item in single_affiliation_list :
    if item not in seen:
        seen.add(item)
        new_list_affi.append(item)
    

In [77]:
#Povoar "PubMed"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(new_list_):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO PubMed (ID_PumMed, title, Doi_number) VALUES (%s, %s, %s)"
        val=(str(new_list_[index]), str(titles[index]), str(doi_list[index]) )
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [7]:
#retirar os valores AI
ID_AI=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_AI_PubMed FROM PubMed"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_= ', '.join(ID_AI)
h_= div_.replace("(",'')
hh_= h_.replace(",)",'')
SEARCH_ID_= hh_.split(', ')
print(SEARCH_ID_)

['19', '20', '21', '22', '23', '24', '25', '26']


In [82]:
#Povação "Gene-Pubmed"

number_map = {}
next_number = int(SEARCH_ID_[0])
relation = []
for number in ID_PUB:
    if number not in number_map:
        number_map[number] = next_number
        next_number += 1
    relation.append(number_map[number])
    
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(id_ncbii):
        
        sql= "INSERT INTO gene_PubMed (ID_genebank, ID_AI_PubMed) VALUES (%s, %s)"
        val=(id_ncbii[index], relation[index])
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [10]:
#Povação "authors"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(new_list_authors):
        
        sql= "INSERT INTO Authors (Name) VALUES (%s)"
        val=(new_list_authors[index],)
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [11]:
#retirar os valores AI de authors
ID_AI_Authors=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_Authors FROM Authors"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI_Authors.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_Authors= ', '.join(ID_AI_Authors)
h_Authors= div_Authors.replace("(",'')
hh_Authors= h_Authors.replace(",)",'')
SEARCH_ID_Authors= hh_Authors.split(', ')
print(SEARCH_ID_Authors)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109']


In [15]:
number_map_ = {}
next_number = int(SEARCH_ID_Authors[0])
output_authors = []
for number in authors_list:
    if number not in number_map_:
        number_map_[number] = next_number
        next_number += 1
    output_authors.append(number_map_[number])

number_map_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub = []
for number in pubmed_list:
    if number not in number_map_pub:
        number_map_pub[number] = next_number
        next_number += 1
    output_pub.append(number_map_pub[number])

#Povação "Pubmed-Authors"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(output_pub):
        
        sql= "INSERT INTO PubMed_Authors (ID_AI_PubMed, ID_Authors) VALUES (%s, %s)"
        val=(str(output_pub[index]),str(output_authors[index]))
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [17]:
#Povação "affi"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(new_list_affi):
        
        sql= "INSERT INTO Affiliation (Info) VALUES (%s)"
        val=(new_list_affi[index],)
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [18]:
#retirar os valores AI de affi
ID_AI_Affiliation=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_Affiliation FROM Affiliation"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI_Affiliation.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_Affiliation= ', '.join(ID_AI_Affiliation)
h_Affiliation= div_Affiliation.replace("(",'')
hh_Affiliation= h_Affiliation.replace(",)",'')
SEARCH_ID_Affiliation= hh_Affiliation.split(', ')
print(SEARCH_ID_Affiliation)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']


In [21]:
number_map_affi_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub_affi = []
for number in pubmed_affi_list:
    if number not in number_map_affi_pub:
        number_map_affi_pub[number] = next_number
        next_number += 1
    output_pub_affi.append(number_map_affi_pub[number])
    
number_map_affi = {}
next_number = 1
output_affi = []
for number in affi_list:
    if number not in number_map_affi:
        number_map_affi[number] = next_number
        next_number += 1
    output_affi.append(number_map_affi[number])




#Povação "Pubmed-Affiliation"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(output_pub_affi):
        
        sql= "INSERT INTO PubMed_Affiliation (ID_AI_PubMed, ID_Affiliation) VALUES (%s, %s)"
        val=(str(output_pub_affi[index]),str(output_affi[index]))
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [ ]:
#Vizualizar tabela History:Erro na escrita na base de dados: 1062 (23000): Duplicate entry '0' for key 'PRIMARY'


import pandas as pd 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)

####### end of connection ####
tabela = pd.read_sql("Select * FROM Gene", DataBase)
DataBase.close()
print(tabela)